## Realtime emotation detection

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import mediapipe as mp

# Load the emotion classification model
model = load_model("G:/zolo/classroom-attention-monitor/models/emotion_model.h5")

# Emotion classes
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# MediaPipe Face Mesh setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=False)
drawing = mp.solutions.drawing_utils

# Webcam
cap = cv2.VideoCapture(0)

IMG_SIZE = 48  # FER-2013 input size

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    label = "No Face Detected"

    if result.multi_face_landmarks:
        face = result.multi_face_landmarks[0]
        lm = face.landmark

        # Get bounding box
        x_coords = [int(l.x * w) for l in lm]
        y_coords = [int(l.y * h) for l in lm]
        x_min, x_max = max(min(x_coords) - 10, 0), min(max(x_coords) + 10, w)
        y_min, y_max = max(min(y_coords) - 10, 0), min(max(y_coords) + 10, h)

        face_img = frame[y_min:y_max, x_min:x_max]

        if face_img.size > 0:
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (IMG_SIZE, IMG_SIZE)) / 255.0
            input_img = resized.reshape(1, IMG_SIZE, IMG_SIZE, 1)

            pred = model.predict(input_img, verbose=0)
            emotion = emotion_labels[np.argmax(pred)]
            label = f"{emotion}"

            # Draw bounding box and label
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)
            cv2.putText(frame, label, (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Show emotion label on frame
    cv2.putText(frame, f"Emotion: {label}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 2)

    cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
